In [1]:
# Importing the important libraries
import pandas as pd 
import numpy as np
import nltk
nltk.download("punkt")
import scipy.sparse as sparse
from pyarabic.araby import strip_tashkeel
import re
import gensim
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
import fasttext.util    
import fasttext
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from keras.utils import pad_sequences
import math

from sklearn import svm
from sklearn.metrics import accuracy_score
from gensim.models import KeyedVectors

from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense,SimpleRNN

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import ISRIStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from keras.preprocessing.text import Tokenizer

from scikeras.wrappers import KerasClassifier

from keras.layers import Dense, Activation, Dropout,Flatten
from keras.callbacks import EarlyStopping
from sklearn.model_selection import GridSearchCV


from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.datasets import make_classification
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
#from torch.optimizer import Adam
from keras.optimizers import Adam, SGD, RMSprop

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lojinkhattab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Reading the data
data= pd.read_csv("preprocessedData100K.csv")
data['fineText']=data['text']
data
data=data.drop(['text','Unnamed: 0','Unnamed: 0.1'], axis=1) 
data

,id,dialect,transformed,fineText
0,1025817321523343360,LB,12,اى والله عم مر صوتى عندك شى للبيع
1,903641435685228416,YE,9,كل عام وانت بخير
2,929851184222961664,LY,1,العقل يا بعاد الدار يمسى معاى وايبات عندكم
3,650866088570449920,OM,14,دائما اقول لما برجع من دوام بنام دائما لما ارج...
4,875454911701540864,SA,8,بعض البنات انا والله ما اكل شى بس اشرب مويه وا...
...,...,...,...,...
99995,1084394265516609536,AE,16,يا ريال يكفى ان نص اعلانات الانستا والسناب عن ...
99996,1150541508426260352,LY,1,بكرا نكمل دورنى فى اى وقت
99997,1129510595437485952,KW,13,انا كنت مهاجم وهداف كنت العب بالقادسيه تحت وحت...
99998,581775425145303040,IQ,0,ايران تهزا وتسخر من القمم العربيه البايخه قمه ...


In [3]:
# Data type of each column
data.dtypes

id              int64
dialect        object
transformed     int64
fineText       object
dtype: object

In [4]:
# Missing values in the data
nullData= data.isnull().sum()
nullData

id              0
dialect         0
transformed     0
fineText       26
dtype: int64

In [5]:
# Dropping the null values in the data
data= data.dropna(axis=0)
# Ensuring that there are no missing values
data.isnull().sum() / data.shape[0] * 100

id             0.0
dialect        0.0
transformed    0.0
fineText       0.0
dtype: float64

In [6]:
# Putting each word in separate column:
words = data['fineText'].str.split()
for i in range(words.str.len().max()):
    data['word{}'.format(i)] = words.str[i]

/var/folders/_m/2b51_hvs78l5wwx7k1rkzlym0000gn/T/ipykernel_680/709653743.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['word{}'.format(i)] = words.str[i]
/var/folders/_m/2b51_hvs78l5wwx7k1rkzlym0000gn/T/ipykernel_680/709653743.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['word{}'.format(i)] = words.str[i]
/var/folders/_m/2b51_hvs78l5wwx7k1rkzlym0000gn/T/ipykernel_680/709653743.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

In [7]:
fasttext.util.download_model('ar', if_exists='ignore')
ft = fasttext.load_model('cc.ar.300.bin')
k=0
for k in range(10):
    column_name = f"word{k}"
    for i in data.index:
        word = data[column_name][i]
        if pd.isna(word):
            # if the word is NaN, replace it with a numpy array of zeros
            data[column_name][i] = np.zeros((300))
        else:
            # if the word is not NaN, embed it using FastText
            word_edit = ft.get_word_vector(word)
            data[column_name][i]=word_edit
     

/var/folders/_m/2b51_hvs78l5wwx7k1rkzlym0000gn/T/ipykernel_680/963597443.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name][i]=word_edit
/var/folders/_m/2b51_hvs78l5wwx7k1rkzlym0000gn/T/ipykernel_680/963597443.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name][i] = np.zeros((300))


In [8]:
k=10
for k in range(10, 21):
    column_name = f"word{k}"
    print(column_name)
    for i in data.index:
        word = data[column_name][i]
        if pd.isna(word):
            # if the word is NaN, replace it with a numpy array of zeros
            data[column_name][i] = np.zeros((300))
        else:
            # if the word is not NaN, embed it using FastText
            word_edit = ft.get_word_vector(word)
            data[column_name][i]=word_edit       


word10


/var/folders/_m/2b51_hvs78l5wwx7k1rkzlym0000gn/T/ipykernel_680/966079164.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name][i] = np.zeros((300))
/var/folders/_m/2b51_hvs78l5wwx7k1rkzlym0000gn/T/ipykernel_680/966079164.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name][i]=word_edit


word11
word12
word13
word14
word15
word16
word17
word18
word19
word20


In [9]:
fasttext.util.download_model('ar', if_exists='ignore')
ft = fasttext.load_model('cc.ar.300.bin')

for k in range(21,31):
    column_name = f"word{k}"
    print(column_name)
    for i in data.index:
        word = data[column_name][i]
        if pd.isna(word):
            # if the word is NaN, replace it with a numpy array of zeros
            data[column_name][i] = np.zeros((300))
        else:
            # if the word is not NaN, embed it using FastText
            word_edit = ft.get_word_vector(word)
            data[column_name][i]=word_edit

/var/folders/_m/2b51_hvs78l5wwx7k1rkzlym0000gn/T/ipykernel_680/3444496518.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name][i] = np.zeros((300))
/var/folders/_m/2b51_hvs78l5wwx7k1rkzlym0000gn/T/ipykernel_680/3444496518.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name][i]=word_edit


word21
word22
word23
word24
word25
word26
word27
word28
word29
word30


In [10]:
fasttext.util.download_model('ar', if_exists='ignore')
ft = fasttext.load_model('cc.ar.300.bin')

for k in range(31,41):
    column_name = f"word{k}"
    print(column_name)
    for i in data.index:
        word = data[column_name][i]
        if pd.isna(word):
            # if the word is NaN, replace it with a numpy array of zeros
            data[column_name][i] = np.zeros((300))
        else:
            # if the word is not NaN, embed it using FastText
            word_edit = ft.get_word_vector(word)
            data[column_name][i]=word_edit

/var/folders/_m/2b51_hvs78l5wwx7k1rkzlym0000gn/T/ipykernel_680/780893581.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name][i] = np.zeros((300))
/var/folders/_m/2b51_hvs78l5wwx7k1rkzlym0000gn/T/ipykernel_680/780893581.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name][i]=word_edit


word31
word32
word33
word34
word35
word36
word37
word38
word39
word40


In [11]:
fasttext.util.download_model('ar', if_exists='ignore')
ft = fasttext.load_model('cc.ar.300.bin')

for k in range(41,51):
    column_name = f"word{k}"
    print(column_name)
    for i in data.index:
        word = data[column_name][i]
        if pd.isna(word):
            # if the word is NaN, replace it with a numpy array of zeros
            data[column_name][i] = np.zeros((300))
        else:
            # if the word is not NaN, embed it using FastText
            word_edit = ft.get_word_vector(word)
            data[column_name][i]=word_edit

/var/folders/_m/2b51_hvs78l5wwx7k1rkzlym0000gn/T/ipykernel_680/1189563619.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name][i] = np.zeros((300))
/var/folders/_m/2b51_hvs78l5wwx7k1rkzlym0000gn/T/ipykernel_680/1189563619.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name][i]=word_edit


word41
word42
word43
word44
word45
word46
word47
word48
word49
word50


In [12]:
fasttext.util.download_model('ar', if_exists='ignore')
ft = fasttext.load_model('cc.ar.300.bin')

for k in range(51,61):
    column_name = f"word{k}"
    print(column_name)
    for i in data.index:
        word = data[column_name][i]
        if pd.isna(word):
            # if the word is NaN, replace it with a numpy array of zeros
            data[column_name][i] = np.zeros((300))
        else:
            # if the word is not NaN, embed it using FastText
            word_edit = ft.get_word_vector(word)
            data[column_name][i]=word_edit

/var/folders/_m/2b51_hvs78l5wwx7k1rkzlym0000gn/T/ipykernel_680/3741161068.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name][i] = np.zeros((300))
/var/folders/_m/2b51_hvs78l5wwx7k1rkzlym0000gn/T/ipykernel_680/3741161068.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name][i]=word_edit


word51
word52
word53
word54
word55
word56
word57
word58
word59
word60


In [224]:
# Defining a function to concatenate the values of the 'word' columns
def combine_columns(row):
    combined_values = []
    for col in data.columns:
        if col.startswith('word'):
            combined_values.append(row[col])
    return combined_values
# Applying the function
data.loc[:100000, 'Sentence'] = data.iloc[:100000].apply(combine_columns, axis=1)


In [226]:
# Ensuring that there is no missing values:
missing_values_mask = data['Sentence'].isna()
missing_values_index = data[missing_values_mask].index
print(missing_values_index.tolist())


[]


In [227]:
# Dropping the words columns
data = data.drop(columns=[col for col in data.columns if col.startswith('word')])

In [232]:
# Saving the embedded data as csv
data.to_csv('embeddings100KCopy2.csv')